## Pygrib NBM Quick Start

Quick look at pygrib and National Blended Model data.

pygrib is quite finicky to install and should often be the first library you install in your environment.  

In [ ]:
import pygrib
import os,glob
import boto3, botocore
import datetime

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
import numpy as np
import ipywidgets as widgets
from metpy.plots import ImagePlot, ContourPlot, MapPanel, PanelContainer

to_proj = ccrs.AlbersEqualArea(central_longitude=-97., central_latitude=38.)

## Change the following line to point to where you would like your files saved locally.

In [ ]:
os.chdir('/path/to/put/downloaded/data')

## The following shape files can be found online here:
Counties: https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html

CWAs: https://www.weather.gov/gis/CWABounds

Depending on the files you download and where you end up putting them, adjust the following relative two paths.

In [ ]:
reader = shpreader.Reader('./cb_2018_us_county_5m/cb_2018_us_county_5m.shp')
counties = list(reader.geometries())
COUNTIES = cfeature.ShapelyFeature(counties, ccrs.PlateCarree())

reader = shpreader.Reader('./w_10nv20/w_10nv20.shp')
cwas = list(reader.geometries())
CWAS = cfeature.ShapelyFeature(cwas, ccrs.PlateCarree())

## The following cell is for extra map features if you need to add them

In [ ]:
#Build out another background or border set here:

#Insert the path to the shapefile here:
#reader = shpreader.Reader('./path/to.shp')

#Name the shape file variable here
#shpFile = list(reader.geometries())

#Load just the necessary features into another variable with a projection here
#NEW_FEATURES = cfeature.ShapelyFeature(shpFile, ccrs.PlateCarree())

#The above line is what you use as your feature in the third to last code cell for adding more features to the map.

## Set times and paths

In [ ]:
dtNow = datetime.datetime.utcnow()
tt = dtNow.timetuple()
print(tt)
year = tt.tm_year
month = int(tt.tm_mon)
month = "{0:0>2}".format(month)
day = int(tt.tm_mday)
day = "{0:0>2}".format(day)

#The following line is set to hour-1 because data tends to not be in the S3 bucket until an hour later.
#If data isn't showing up, you can push this back further in time to get data.
#It is also possible that you have recently crossed over 00 UTC and this calculation is no longer accurate.
hour = int(tt.tm_hour)-1 
hour = "{0:0>2}".format(hour)

#This is the forecast hour you would like to download up to.  
fHour = 84
fHourLimit = "{0:0>3}".format(fHour)

#If you need a specific date, use the following lines of code and change the dates and times accordingly.
#year = 2022
#month = 2
#month = "{0:0>2}".format(month)
#day = 1
#day = "{0:0>2}".format(day)
#hour = 16
#hour = "{0:0>2}".format(hour)

#noaa-nbm-grib2-pds is National Blend of Models products
bucketName = "noaa-nbm-grib2-pds"

#In theory GFS data is also available in AWS... 
#you could try to change the bucket name and 
#adjusting the file prefixes in the next code block 
#to see if you can get GFS data to flow

pathName = './data/' + str(year) + str(month) + str(day) + '_' + str(hour) + 'Z'

## Grab a list of available GRIB files from S3 bucket
AWS S3 bucket access using boto3 requires the use of credentials within an environment variable in your command line.  First you will need to download the AWS CLI, then find more information about how to set those credentials.  I haven't done it in years, so I am not exactly sure how to do it, but there is info out there about this. 

In [ ]:
s3 = boto3.resource('s3')
bucket = s3.Bucket(bucketName)
paths = []
filenames = []

#Use for core products (non-QMD)
for obj in bucket.objects.filter(Prefix="blend." + str(year) + str(month) + str(day) + "/" + str(hour) + 
                                 "/core/blend.t" + str(hour) + "z.core").all():

#Use the following two lines for QMD products.  Comment out the previous two code lines.
#for obj in bucket.objects.filter(Prefix="blend." + str(year) + str(month) + str(day) + "/" + str(hour) + 
#                                 "/core/blend.t" + str(hour) + "z.core").all():
    if obj.key.find('.idx') == -1:
        if obj.key.find('.co.') != -1:
            paths.append(obj.key)
            filename = obj.key.split('/')[-1]
            filenames.append(filename)
            if obj.key.find('.f'+fHourLimit) != -1:
                break

## If the folder you will be placing these data files in already exists, then keep moving.

In [ ]:
if (os.path.isdir(pathName)):
    print("Path exists... continuing")
else:
    os.mkdir(pathName)

## For all the files in the S3 bucket up to the forecast hour limit, loop through and download them.

In [ ]:
fList = []
for i, file in enumerate(filenames):
    fList.append(file.split('f')[1].split('.')[0])
    if (int(fList[i]) > fHour):
        fList.pop()
        break

for idx, filename in enumerate(filenames):
    if os.path.isfile(pathName + "/" + filename):
        print("File exists... continuing")
        continue            
    else:            
        print("downloading " + str(fList[idx]) + " of " + str(fList[-1]))
        try:
           bucket.download_file(paths[idx], pathName + "/" + filename)
        except botocore.exceptions.ClientError as e:
           if e.response['Error']['Code'] == "404":
               print("The object does not exist.")
           else:
               raise

## Use ```glob``` to grab the local paths and names of all the NBM files

In [ ]:
grb_path = pathName + "/"
grib_list = sorted(glob.glob(f"{grb_path}*.grib2"))
print(grib_list)
grbs = pygrib.open(grib_list[0])
grb = grbs.read()
print(grb)

## Set up widgets

In [ ]:
#Variable Dropdown
var_widget = widgets.Dropdown(options=grb, description='Variable', alignment='center')

#Colormap Dropdown and reverse scale checkbox
colormap_widget = widgets.Dropdown(description='Colormap',
                                       options=['viridis', 'plasma', 'inferno', 'magma', 'cividis',
                                                 'Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
                                                 'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
                                                 'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn',
                                                 'binary', 'gist_yarg', 'gist_gray', 'gray', 'bone', 'pink',
                                                 'spring', 'summer', 'autumn', 'winter', 'cool', 'Wistia',
                                                 'hot', 'afmhot', 'gist_heat', 'copper',
                                                 'PiYG', 'PRGn', 'BrBG', 'PuOr', 'RdGy', 'RdBu',
                                                 'RdYlBu', 'RdYlGn', 'Spectral', 'coolwarm', 'bwr', 'seismic',
                                                 'twilight', 'twilight_shifted', 'hsv', 
                                                 'Pastel1', 'Pastel2', 'Paired', 'Accent',
                                                 'Dark2', 'Set1', 'Set2', 'Set3',
                                                 'tab10', 'tab20', 'tab20b', 'tab20c',
                                                 'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
                                                 'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg',
                                                 'gist_rainbow', 'rainbow', 'jet', 'turbo', 'nipy_spectral',
                                                 'gist_ncar'])
reverse_colormap_widget = widgets.Checkbox(value=False, description='Reverse Colorscale', 
                                           disabled=False, indent=True)

#colormap max, min, and contour count
colormap_max_widget = widgets.FloatText(description="maxVal", value=100, continuous_update=False)
colormap_min_widget = widgets.FloatText(description="minVal", value=0, continuous_update=False)
colormap_contour_widget = widgets.BoundedIntText(description="# of contours", value=10, min=1, max=100, 
                                                 continuous_update=False)

#map controls
map_lat_widget = widgets.SelectionRangeSlider(options=np.arange(-90,91,1), description='Latitude', 
                                              value=[18,55], disabled=False, continuous_update=False)
map_lon_widget = widgets.SelectionRangeSlider(options=np.arange(-180,181,1), description='Longitude', 
                                              value=[-140,-58], disabled=False, continuous_update=False)

#time slider
time_widget = widgets.SelectionSlider(options=fList, 
                                      description='fhour:', 
                                      disabled=False,
                                      continuous_update=False, 
                                      orientation='horizontal', 
                                      readout=True, 
                                      readout_format='d')   

#Other widgets that could be added
# -- Controls to turn on and off counties and CWAs
# -- Controls to control the colors of each of the borders
# -- Controls to use max and min from the dataset to control the colorscale

## Function to plot data on-the-fly

In [ ]:
def plot(varname='', colormap='', reverse='False', maxval='100', minval='0', conts='1', map_lat='', map_lon='', time='001'):      

#Get the grib message number that corresponds to the selected variable
    varNum = str(varname).split(':')
#Convert to array numbers (0-based)
    arrNum = int(varNum[0])-1

#Get the time index from within the fList[...fHours...]
    timeIndex = fList.index(time)

#Open that grb file for reading
    grbs = pygrib.open(grib_list[timeIndex])

#Read that grb file
    grb = grbs.read()

#Select just the variable needed and pull out the data values, the lat, and the lon.
    grbData = grb[arrNum]
    nbm,lat,lon = grbData.data() # -> gets all the plot data in one call
    
#Get names of the variable and fHour for titling.
    varShortName = grbData.shortName
    varLongName = grbData.name
    fHour = "{0:0>3}".format(time)
    nbmMax = np.max(nbm)
    nbmMin = np.min(nbm)
    
#Set up plot sizes
    fig = plt.figure(figsize=(20, 10))
    ax = fig.add_subplot(1,1,1, projection=ccrs.PlateCarree())
    ax.set_extent([map_lon[0], map_lon[1], map_lat[0], map_lat[1]])
    
#Set title on figure.
    ax.set_title(str(varLongName)+" ("+str(varShortName)+'; max='+str(nbmMax)+', min='+str(nbmMin)+
                 ') fHour: ' + str(fHour), size=16)

#if colormap needs to be reversed
    if reverse==True:
        colormap=colormap + "_r"
    
# Contour based on variable chosen
    c = ax.contourf(lon, lat, grbData.values, 
                    np.arange(minval, maxval, (maxval-minval)/conts), 
                    cmap=colormap, transform=ccrs.PlateCarree())
    cb = fig.colorbar(c, ax=ax, shrink=0.7, boundaries=[minval, maxval])
        
# Add country/coastline, counties, CWA, and state boundaries to the plot
    ax.add_feature(cfeature.BORDERS)
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(COUNTIES, facecolor='none', edgecolor='lightsteelblue')
    ax.add_feature(CWAS, facecolor='none', edgecolor='gray')
    ax.add_feature(cfeature.STATES, edgecolor='black')
    
#If you need more borders or map backgrounds, 
#you can replace the "NEW_FEATURES" in the below line with your variable
#Build the variable out in the fourth code cell at the top of this notebook.
#    ax.add_feature(NEW_FEATURES, facecolor='none', edgecolor='lightsteelblue')

In [ ]:
#Function to update the listed variables since each grib file has a different listing of variables.
def updateVar(sender, time=''):
    var_val = var_widget.value
    var_split = str(var_val).split(':')

    time = time_widget.value
    
    count = 0
    for check in var_split:
        count+=1
        if 'fcst' in check:
            var_val = ':'.join(var_split[1:count-1])
            break

    timeIndex = fList.index(time)
    grbs = pygrib.open(grib_list[timeIndex])
    grb = grbs.read()
    
    matches = []
    for match in grb:
        if var_val in str(match):
            matches.append(str(match))
    
    var_widget.options = grb
    if len(matches):
        var_widget.index = int(matches[0].split(':')[0])-1
    else: 
        var_widget.index = 1
time_widget.observe(updateVar, names=['value'])

## Build the Interactive Widgets into a viewer

In [ ]:
x = widgets.interactive(plot, varname=var_widget, colormap=colormap_widget, reverse=reverse_colormap_widget,
                        maxval=colormap_max_widget, minval=colormap_min_widget, conts=colormap_contour_widget, 
                        map_lat=map_lat_widget, map_lon=map_lon_widget,
                        time=time_widget)
display(x)

### For a partial list of variable transformations from grib list to NBM variable go here:
https://docs.google.com/spreadsheets/d/1z9-ASxixoCA2SmCsWmdfAs1s2AY5kLxTPPIrkhUal1k/edit?usp=sharing

### To see a visual listing of colorscales, go here:
https://matplotlib.org/stable/tutorials/colors/colormaps.html